In [1]:
import pandas as pd

### Sentiment analysis of tweet data

#### Load data

In [2]:
tweet_data = pd.read_csv('tweet_data.csv')
tweet_data.head()

,Company,Date,Tweet,Followers
0,VZ,2014-10-14,news actives on open aapl u2026 board view,186
1,VZ,2015-06-20,pswseeking alpha june trade review,155
2,VZ,2015-06-20,pswseeking alpha june trade review,87
3,VZ,2015-06-20,pswseeking alpha june trade review,603
4,VZ,2015-06-20,pswseeking alpha june trade review,32


#### Get sentiment intensity of each tweet

In [3]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

senti_pol = []
for i in range(len(tweet_data)):
    tmp = str( tweet_data.iloc[i,2] )
    pol = sid.polarity_scores(tmp)
    senti_pol.append(pol['compound'])
    
tweet_data['SentimentPolarity'] = senti_pol
tweet_data.describe()

,Followers,SentimentPolarity
count,119844.000000,119844.000000
mean,1039.686250,0.086797
std,7369.958122,0.273342
min,-2.000000,-0.959300
25%,16.000000,0.000000
50%,198.000000,0.000000
75%,590.000000,0.202300
max,534214.000000,0.953800


#### Create a key - 'ComDate' - to later merge with stock data

In [4]:
tweet_data['ComDate'] = tweet_data['Company'] + '-' + tweet_data['Date']
tweet_data.head()

,Company,Date,Tweet,Followers,SentimentPolarity,ComDate
0,VZ,2014-10-14,news actives on open aapl u2026 board view,186,0.2732,VZ-2014-10-14
1,VZ,2015-06-20,pswseeking alpha june trade review,155,0.0000,VZ-2015-06-20
2,VZ,2015-06-20,pswseeking alpha june trade review,87,0.0000,VZ-2015-06-20
3,VZ,2015-06-20,pswseeking alpha june trade review,603,0.0000,VZ-2015-06-20
4,VZ,2015-06-20,pswseeking alpha june trade review,32,0.0000,VZ-2015-06-20


#### Normalize follower count

In [5]:
tweet_data['Followers'] = (tweet_data['Followers'] - tweet_data['Followers'].min())/(tweet_data['Followers'].max() - tweet_data['Followers'].min())
tweet_data.describe()

,Followers,SentimentPolarity
count,119844.000000,119844.000000
mean,0.001950,0.086797
std,0.013796,0.273342
min,0.000000,-0.959300
25%,0.000034,0.000000
50%,0.000374,0.000000
75%,0.001108,0.202300
max,1.000000,0.953800


#### Calculate new parameter 'effectiveSentimentPolarity'  
#### effective sentiment polarity = Sum{ (normalized follower count) * (sentiment polarity) } / (No. of tweets)

In [6]:
tweet_data['FollowerSentimentProduct'] = tweet_data['Followers'] * tweet_data['SentimentPolarity']
tweet_data.head()

,Company,Date,Tweet,Followers,SentimentPolarity,ComDate,FollowerSentimentProduct
0,VZ,2014-10-14,news actives on open aapl u2026 board view,0.000352,0.2732,VZ-2014-10-14,0.000096
1,VZ,2015-06-20,pswseeking alpha june trade review,0.000294,0.0000,VZ-2015-06-20,0.000000
2,VZ,2015-06-20,pswseeking alpha june trade review,0.000167,0.0000,VZ-2015-06-20,0.000000
3,VZ,2015-06-20,pswseeking alpha june trade review,0.001133,0.0000,VZ-2015-06-20,0.000000
4,VZ,2015-06-20,pswseeking alpha june trade review,0.000064,0.0000,VZ-2015-06-20,0.000000


In [7]:
EffectiveSentimentPolarity = tweet_data.groupby(['ComDate']).mean()*100
EffectiveSentimentPolarity.describe()

,Followers,SentimentPolarity,FollowerSentimentProduct
count,29250.000000,29250.000000,29250.000000
mean,0.174528,8.851194,0.015217
std,0.761225,21.201620,0.220277
min,0.000374,-93.600000,-10.788834
25%,0.009640,0.000000,0.000000
50%,0.064535,0.000000,0.000000
75%,0.136178,19.090000,0.006049
max,36.851573,93.490000,15.532938


#### Create new dataframe

In [8]:
new_tweet_data = {
    'ComDate': EffectiveSentimentPolarity.index,
    'EffectiveSentimentPolarity': EffectiveSentimentPolarity['FollowerSentimentProduct'].values
}

new_tweet_data = pd.DataFrame(new_tweet_data, index=range(len(EffectiveSentimentPolarity)))
new_tweet_data.head()

,ComDate,EffectiveSentimentPolarity
0,AAPL-2014-01-01,-0.088290
1,AAPL-2014-01-02,0.002490
2,AAPL-2014-01-03,0.001807
3,AAPL-2014-01-04,0.022700
4,AAPL-2014-01-05,0.002168


#### Write dataframe to file

In [9]:
new_tweet_data.to_csv('sentiment_data.csv', index=False)

### Stock data

#### Load data

In [10]:
stock_data = pd.read_csv('stock_data.csv')
stock_data.head()

,Company,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2012-09-04,95.108574,96.448570,94.928574,96.424286,87.121140,91973000.0
1,AAPL,2012-09-05,96.510002,96.621429,95.657143,95.747147,86.509338,84093800.0
2,AAPL,2012-09-06,96.167145,96.898575,95.828575,96.610001,87.288956,97799100.0
3,AAPL,2012-09-07,96.864288,97.497147,96.538574,97.205711,87.827171,82416600.0
4,AAPL,2012-09-10,97.207146,97.612854,94.585716,94.677139,85.542564,121999500.0


#### Cross verify no. of companies

In [11]:
print( 'No. of companies in Tweet data = ' + str( len(tweet_data['Company'].unique()) ) )
print( 'No. of companies in Stock data = ' + str( len(stock_data['Company'].unique()) ) )
print( str( set(stock_data['Company'].unique()) - set(tweet_data['Company'].unique()) ) + ' is the extra company in Stock data' )

# discarding GMRE data from stock_data
stock_data.drop( stock_data[stock_data['Company'] == 'GMRE'].index, inplace=True)
print( 'No. of companies in Stock data = ' + str( len(stock_data['Company'].unique()) ) )

No. of companies in Tweet data = 87
No. of companies in Stock data = 88
{'GMRE'} is the extra company in Stock data
No. of companies in Stock data = 87


#### Create a key - 'ComDate' - to merge with tweet data

In [12]:
stock_data['ComDate'] = stock_data['Company'] + '-' + stock_data['Date']
stock_data.head()

,Company,Date,Open,High,Low,Close,Adj Close,Volume,ComDate
0,AAPL,2012-09-04,95.108574,96.448570,94.928574,96.424286,87.121140,91973000.0,AAPL-2012-09-04
1,AAPL,2012-09-05,96.510002,96.621429,95.657143,95.747147,86.509338,84093800.0,AAPL-2012-09-05
2,AAPL,2012-09-06,96.167145,96.898575,95.828575,96.610001,87.288956,97799100.0,AAPL-2012-09-06
3,AAPL,2012-09-07,96.864288,97.497147,96.538574,97.205711,87.827171,82416600.0,AAPL-2012-09-07
4,AAPL,2012-09-10,97.207146,97.612854,94.585716,94.677139,85.542564,121999500.0,AAPL-2012-09-10


#### Create temporary dataframes with 'ComDate' as index

In [13]:
sd = pd.DataFrame(stock_data.values, columns=stock_data.columns, index=stock_data['ComDate'])
td = pd.DataFrame(new_tweet_data.values, columns=new_tweet_data.columns, index=new_tweet_data['ComDate'])

In [14]:
combined_data = pd.concat([sd, td], axis=1)
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115555 entries, AAPL-2012-09-04 to XOM-2016-03-25
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Company                     108295 non-null  object
 1   Date                        108295 non-null  object
 2   Open                        108292 non-null  object
 3   High                        108292 non-null  object
 4   Low                         108292 non-null  object
 5   Close                       108292 non-null  object
 6   Adj Close                   108292 non-null  object
 7   Volume                      108292 non-null  object
 8   ComDate                     108295 non-null  object
 9   ComDate                     29250 non-null   object
 10  EffectiveSentimentPolarity  29250 non-null   object
dtypes: object(11)
memory usage: 10.6+ MB


In [15]:
combined_data.head()

,Company,Date,Open,High,Low,Close,Adj Close,Volume,ComDate,ComDate,EffectiveSentimentPolarity
ComDate,,,,,,,,,,,
AAPL-2012-09-04,AAPL,2012-09-04,95.108574,96.44857,94.928574,96.424286,87.12114,91973000.0,AAPL-2012-09-04,NaN,NaN
AAPL-2012-09-05,AAPL,2012-09-05,96.510002,96.621429,95.657143,95.747147,86.509338,84093800.0,AAPL-2012-09-05,NaN,NaN
AAPL-2012-09-06,AAPL,2012-09-06,96.167145,96.898575,95.828575,96.610001,87.288956,97799100.0,AAPL-2012-09-06,NaN,NaN
AAPL-2012-09-07,AAPL,2012-09-07,96.864288,97.497147,96.538574,97.205711,87.827171,82416600.0,AAPL-2012-09-07,NaN,NaN
AAPL-2012-09-10,AAPL,2012-09-10,97.207146,97.612854,94.585716,94.677139,85.542564,121999500.0,AAPL-2012-09-10,NaN,NaN


In [16]:
combined_data.fillna(0, inplace=True)
combined_data.head()

,Company,Date,Open,High,Low,Close,Adj Close,Volume,ComDate,ComDate,EffectiveSentimentPolarity
ComDate,,,,,,,,,,,
AAPL-2012-09-04,AAPL,2012-09-04,95.108574,96.448570,94.928574,96.424286,87.121140,91973000.0,AAPL-2012-09-04,0,0.0
AAPL-2012-09-05,AAPL,2012-09-05,96.510002,96.621429,95.657143,95.747147,86.509338,84093800.0,AAPL-2012-09-05,0,0.0
AAPL-2012-09-06,AAPL,2012-09-06,96.167145,96.898575,95.828575,96.610001,87.288956,97799100.0,AAPL-2012-09-06,0,0.0
AAPL-2012-09-07,AAPL,2012-09-07,96.864288,97.497147,96.538574,97.205711,87.827171,82416600.0,AAPL-2012-09-07,0,0.0
AAPL-2012-09-10,AAPL,2012-09-10,97.207146,97.612854,94.585716,94.677139,85.542564,121999500.0,AAPL-2012-09-10,0,0.0


#### Create a new dataframe with combined data values

In [17]:
final_data = pd.DataFrame(combined_data.values, columns=combined_data.columns).drop(['ComDate'], axis=1)
final_data.head()

,Company,Date,Open,High,Low,Close,Adj Close,Volume,EffectiveSentimentPolarity
0,AAPL,2012-09-04,95.108574,96.44857,94.928574,96.424286,87.12114,91973000.0,0.0
1,AAPL,2012-09-05,96.510002,96.621429,95.657143,95.747147,86.509338,84093800.0,0.0
2,AAPL,2012-09-06,96.167145,96.898575,95.828575,96.610001,87.288956,97799100.0,0.0
3,AAPL,2012-09-07,96.864288,97.497147,96.538574,97.205711,87.827171,82416600.0,0.0
4,AAPL,2012-09-10,97.207146,97.612854,94.585716,94.677139,85.542564,121999500.0,0.0


In [18]:
final_data = final_data.drop(['Date', 'Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)

#### Add timesteps to data

In [19]:
def series_to_supervised(data, colnames, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(colnames[j]+'(t-%d)' % (i)) for j in range(len(colnames))]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(colnames[j]+'(t)') for j in range(len(colnames))]
		else:
			names += [(colnames[j]+'(t+%d)' % (i)) for j in range(len(colnames))]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [20]:
reframed = series_to_supervised(final_data.values, final_data.columns, 30, 1)
reframed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115525 entries, 30 to 115554
Data columns (total 93 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   Company(t-30)                     115525 non-null  object
 1   Close(t-30)                       115525 non-null  object
 2   EffectiveSentimentPolarity(t-30)  115525 non-null  object
 3   Company(t-29)                     115525 non-null  object
 4   Close(t-29)                       115525 non-null  object
 5   EffectiveSentimentPolarity(t-29)  115525 non-null  object
 6   Company(t-28)                     115525 non-null  object
 7   Close(t-28)                       115525 non-null  object
 8   EffectiveSentimentPolarity(t-28)  115525 non-null  object
 9   Company(t-27)                     115525 non-null  object
 10  Close(t-27)                       115525 non-null  object
 11  EffectiveSentimentPolarity(t-27)  115525 non-null  object
 12  C

#### Filter data

In [25]:
filtered_data = reframed.drop(reframed[reframed['Company(t)'] != reframed['Company(t-1)']].index)
for i in range(1,30):
    filtered_data = filtered_data.drop(filtered_data[filtered_data['Company(t)'] != filtered_data['Company(t-%d)'%i]].index)
    filtered_data = filtered_data.drop(['Company(t-%d)'%i], axis=1)
len(filtered_data)

113002

In [26]:
# to_drop = ['Company(t-3)', 'Date(t-3)',
#        'Company(t-2)', 'Date(t-2)',
#        'Company(t-1)', 'Date(t-1)',
#        'Date(t)', 'Open(t)', 'High(t)', 'Low(t)', 'Close(t)',
#        'Volume(t)', 'EffectiveSentimentPolarity(t)']

# filtered_data = filtered_data.drop(to_drop, axis=1)
# filtered_data.head()

#### Write to file

In [27]:
filtered_data.to_csv('aggregate_data.csv', index=False)